In [ ]:
# 네이버의 박은정 박사가 2015년에 발표한 “Naver Sentiment Movie Corpus v1.0"을 이용해 긍정/부정 감성 분석을 해보자.
# 여기에는 훈련 데이터로 15만개, 테스트 데이터로 5만개로 총 20만개의 리뷰가 있다.
import tensorflow as tf
# import numpy as np
# import pandas as pd

# #파일 다운로드
# #urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# #urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

# #파일 매칭
# path_to_train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
# path_to_test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

# # 파일 주입
# train_data = pd.read_table('ratings_train.txt')
# test_data = pd.read_table('ratings_test.txt')

# print(train_data[:5]) # 상위 5개 출력
# print('훈련용 리뷰 개수 :',len(train_data))

# train_text = open(path_to_train_file, 'rb').read().decode(encoding='utf-8')
# test_text = open(path_to_test_file, 'rb').read().decode(encoding='utf-8')

# # 텍스트가 총 몇 자인지 확인
# print('Length of text: {} characters'.format(len(train_text)))
# print(('Length of text: {} characters'.format(len(test_text))))

# # 처음부터 500자를 확인 
# print(train_text[:500])

In [1]:
!pip uninstall tensorflow 

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /Users/etlers/opt/anaconda3/bin/estimator_ckpt_converter
    /Users/etlers/opt/anaconda3/bin/import_pb_to_tensorboard
    /Users/etlers/opt/anaconda3/bin/saved_model_cli
    /Users/etlers/opt/anaconda3/bin/tensorboard
    /Users/etlers/opt/anaconda3/bin/tf_upgrade_v2
    /Users/etlers/opt/anaconda3/bin/tflite_convert
    /Users/etlers/opt/anaconda3/bin/toco
    /Users/etlers/opt/anaconda3/bin/toco_from_protos
    /Users/etlers/opt/anaconda3/lib/python3.9/site-packages/tensorflow-2.8.0.dist-info/*
    /Users/etlers/opt/anaconda3/lib/python3.9/site-packages/tensorflow/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
